# 6. 내장 모듈


* 파이썬은 언어를 사용할 때 일반적으로 사용하는 가장 중요한 모듈은 표준 라이브러리에 갖춰 기본 설치로 제공하고자 함.
* 몇몇 기본적인 내장 모듈은 파이썬의 특징과 밀접한 관련이 있어 복잡하거나 오류가 발생하기 쉬운 프로그램 작성에 특히 중요함.

## Better Way 42. functools.wraps로 함수 데코레이터를 정의하자

* **데코레이터(decorator)**
    - 파이썬에서 함수에 적용할 수 있는 특별한 문법.
    - 감싸고 있는 함수를 호출하기 전 또는 후에 추가로 코드를 실행하는 기능을 가짐.
        + 입력 인수 및 반환 값에 대한 접근과 수정이 가능.
        + 시맨틱 강조, 디버깅, 함수 등록 등 여러 상황에 유용.

* [예시] 함수를 호출할 때 인수와 반환 값을 출력하기.
    - 특히 재귀 호출에서 함수 호출의 스택을 디버깅할 때 도움이 됨.
    - 아래와 같이 데코레이터를 정의.

In [5]:
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r' %
              (func.__name__, args, kwargs, result))
        return result
    return wrapper

* @ 기호로 이 데코레이터를 함수에 적용.

In [6]:
@trace
def fibonacci(n):
    """n번째 피보나치 수를 반환한다."""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

* @ 기호는 감싸고 있는 함수를 인수로 사용하여 해당 데코레이터를 호출한 후 반환 값을 같은 스코프에 있는 원래 이름에 할당하는 코드에 상응.

In [7]:
# fibonacci = trace(fibonacci)

* 이 데코레이터 함수를 호출하면 `fibonacci` 실행 전후에 `wrapper` 코드를 실행해 재귀 스택의 각 단계마다 인수와 반환 값을 출력함.

In [8]:
fibonacci(3)

fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2


2

* 코드는 잘 동작하지만 의도치 않은 부작용을 일으킴.
    - 데코레이터에서 반환한 값(앞에서 호출한 함수)의 이름이 fibonacci가 아님.

In [9]:
print(fibonacci)

<function trace.<locals>.wrapper at 0x7fd70190c730>


* 원인?
    - `trace` 함수는 그 안에 정의된 `wrapper`를 반환.
    - `wrapper` 함수가 데코레이터를 호출한 후 해당 호출을 담고 있는 모듈의 `fibonacci`라는 이름에 할당되는 값임.
    - 이 동작은 디버거(**BW57 참조**), 객체 직렬화 기능(**BW44 참조**) 등 객체 내부를 조사하는 도구를 사용할 때 문제가 될 수 있음.
* [예시] 데코레이터를 적용한 `fibonacci` 함수에는 내장 함수 `help`가 쓸모없어짐.

In [10]:
# help(fibonacci)

* 아래처럼 출력된다고 함.

    Help on function wrapper in module __main__:
    
    wrapper(*args, **kwargs)

* 해결책?
    - **내장 모듈 `functools`의 `wraps` 헬퍼 함수 사용.**
		+ **`wraps`**
            + 데코레이터를 작성하는 데 이용하는 데코레이터.
            + `wrapper` 함수에 적용하면 내부 함수에 있는 중요 메타데이터가 모두 외부 함수로 복사됨.

In [11]:
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r' %
              (func.__name__, args, kwargs, result))
        return result
    return wrapper

@trace
def fibonacci(n):
    """n번째 피보나치 수를 반환한다."""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

In [12]:
help(fibonacci)

Help on function fibonacci in module __main__:

fibonacci(n)
    n번째 피보나치 수를 반환한다.



* 이렇게 하면 대상 함수에 데코레이터를 적용했더라도 원하는 `help` 결과가 나옴.

* `wraps`의 유용성
    - `help`를 호출한 예는 데코레이터가 어떤 식으로 미묘한 문제를 일으키는지 보여주는 사례 중 하나.
    - 파이썬 함수에는 여러 표준 속성(`__name__`, `__module__`)이 있으며, 언어에서 함수들의 인터페이스를 유지하려면 이 속성들을 반드시 보호해야 함.
    - `wraps`를 사용하면 항상 올바른 동작을 얻을 수 있음.


### 핵심 정리
* 데코레이터는 런타임에 한 함수로 다른 함수를 수정할 수 있게 해주는 파이썬 문법이다.
* 데코레이터를 사용하면 디버거와 같이 객체 내부를 조사하는 도구가 이상하게 동작할 수도 있다.
* 직접 데코레이터를 정의할 때 이런 문제를 피하려면 내장 모듈 `functools`의 `wraps` 데코레이터를 사용하자.